In [21]:
import pandas as pd
import re

In [22]:
# topic = 'mentioned_topic'
topic = 'against_topic'

# issue = 'issue224_allCommentsText.doc'
# issue = 'issue396_allCommentsText.doc'
# issue = 'issue4865_allCommentsText.doc'
# issue = 'issue9388_allCommentsText.doc'
issue = 'issue14909_allCommentsText.doc'

In [23]:
df_all = pd.read_csv('Preprocessed.csv')
booleanDictionary = {'True': True, 'False': False}
df_all.replace(booleanDictionary, inplace=True)

In [24]:
def GetMentionedTopic(row):
    argu_part_code = row['argu_part']
    code_str = row['Codes']
    idx_list = []
    code_len = 0
    if argu_part_code == 1: # claim
        idx_list = [m.start() for m in re.finditer('claim', code_str)]
        code_len = len('claim')
    elif argu_part_code == 2: # warrant
        idx_list = [m.start() for m in re.finditer('warrant', code_str)]
        code_len = len('warrant')
    elif argu_part_code == 3: # data
        idx_list = [m.start() for m in re.finditer('ground', code_str)]
        code_len = len('ground')
    claim_topic = ''
    if len(idx_list) == 0:
        claim_topic = '0,'
    else:
        for idx in idx_list:
            end_idx = code_str.find(',', idx+code_len+1)
            if end_idx == -1:
                claim_topic += code_str[idx+code_len+1: ] + ','
            else:
                claim_topic += code_str[idx+code_len+1: end_idx] + ','
    return claim_topic[:-1]

In [25]:
def GetAgainstTopic(row):
    is_rebuttal_code = row['is_against']
    code_str = row['Codes']
    idx_list = []
    code_len = 0
    if is_rebuttal_code == 1: # rebuttal
        idx_list = [m.start() for m in re.finditer('against', code_str)]
        code_len = len('against')
    rebuttal_topic = ''
    if len(idx_list) == 0:
        rebuttal_topic = '0,'
    else:
        for idx in idx_list:
            end_idx = code_str.find(',', idx+code_len+1)
            if end_idx == -1:
                rebuttal_topic += code_str[idx+code_len+1: ] + ','
            else:
                rebuttal_topic += code_str[idx+code_len+1: end_idx] + ','
    return rebuttal_topic[:-1]

In [26]:
if topic == 'mentioned_topic':
    df_all[topic]  = df_all.apply(GetMentionedTopic, axis=1)
elif topic == 'against_topic':
    df_all[topic]  = df_all.apply(GetAgainstTopic, axis=1)

In [27]:
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    row_accumulator = []

    def splitListToRows(row, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)

    df.apply(splitListToRows, axis=1, args = (separator, ))
    new_df = pd.DataFrame(row_accumulator)
    return new_df

In [28]:
df_topic_splited = splitDataFrameList(df_all, topic, ',')

In [29]:
df_topic_splited.Document.value_counts()

issue224_allCommentsText.doc      2039
issue396_allCommentsText.doc      1856
issue14909_allCommentsText.doc     621
issue4865_allCommentsText.doc      354
issue9388_allCommentsText.doc      285
Name: Document, dtype: int64

In [30]:
df_issue_splited = df_topic_splited.loc[df_topic_splited['Document'] == issue].copy()

In [31]:
df_issue_splited[topic] = df_issue_splited[topic].astype(int)

In [32]:
pd.options.display.max_rows = 4000

display(df_issue_splited[topic].value_counts().sort_index())

0     615
1       2
4       3
14      1
Name: against_topic, dtype: int64